In [1]:
import os, sys, re, pickle
import pandas as pd
import numpy as np

from tqdm import tqdm

from collections import Counter

import os.path as osp

datafolderpath = "../dataset"
train_csv = "train.csv"
valid_csv = "valid.csv"
train_human_pkl = "train_human_generated.pkl"
valid_human_pkl = "valid_human_generated.pkl"

In [2]:
traindf = pd.read_csv(osp.join(datafolderpath, train_csv))
train_hum_gen_map = pickle.load(open(osp.join(datafolderpath, train_human_pkl), 'rb'))
hum_generated_column = []

for ind, row in traindf.iterrows():
    hum_generated_column.append(train_hum_gen_map[row["English"]])

traindf["hum_gen"] = hum_generated_column

traindf.shape

(2766, 6)

In [3]:
valdf = pd.read_csv(osp.join(datafolderpath, valid_csv))
val_hum_gen_map = pickle.load(open(osp.join(datafolderpath, valid_human_pkl), 'rb'))

hum_generated_column = []

for ind, row in valdf.iterrows():
    hum_generated_column.append(val_hum_gen_map[row["English"]])

valdf["hum_gen"] = hum_generated_column
    
valdf.shape

(395, 4)

In [4]:
traindf.head()

,English,Hindi,Hinglish,Average rating,Disagreement,hum_gen
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6,"[Program module ek sanchika hoti hai, jisme ya..."
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4,[Aur humne unke bhai Salih ko Thamud ke paas b...
2,"and, when reminded, do not remember\n","और जब उन्हें याद दिलाया जाता है, तो वे याद नही...","aur jab unhen yad dilaya jata hai , to ve yad ...",10,0,"[And jab remind karaya jaata hai, then they do..."
3,you won the TED Prize 2011.\n,तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n,tumhen २०११ ka ted prize mil gaya hai\n,9,1,"[tumhe TED Prize 2011 mil gaya hai.\n, You won..."
4,He gone to Kerodemal College of Delhi Universi...,उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...,unhonne bad science karne ke lie ye delhi univ...,7,0,[Unhone baad adhyayan karne ke liye ye Delhi U...


# CSNLI - [GitHub Repo](https://github.com/devanshg27/csnli)

- for script normalisation and LID

In [2]:
os.chdir('/home2/anmol.goel/prashantk/csnli')

print(os.getcwd())
from three_step_decoding import *


tsd = ThreeStepDecoding('/home2/anmol.goel/prashantk/csnli/lid_models/hinglish', 
                        htrans='/home2/anmol.goel/prashantk/csnli/nmt_models/rom2hin.pt', 
                        etrans='/home2/anmol.goel/prashantk/csnli/nmt_models/eng2eng.pt')

#testing if csnli tool is working
print('\n'.join(['\t'.join(x) for x in tsd.tag_sent(u'i thght mosam dfrnt hoga bs fog h')]))

/home2/anmol.goel/prashantk/csnli
Loading NMT model parameters.
Loading NMT model parameters.


RuntimeError: index_select(): Expected dtype int32 or int64 for index

In [ ]:
line = u'i thght mosam dfrnt hoga bs fog h'
list(tsd.tag_sent(line))

In [40]:
def normalise_sent(sent):
    try:
        ret = list(tsd.tag_sent(sent))
    except TypeError:
        ret = [sent]
        print(f"TypeError : {sent}")
    return ret
              
def normalise_df_column(df, column_name):
    csnli_op = []
    if isinstance(df[column_name][0], str):
        csnli_op = [normalise_sent(sent) for sent in tqdm(df[column_name])]
        
    elif isinstance(df[column_name][0], list):
        csnli_op = [[normalise_sent(sent) for sent in sample] for sample in tqdm(df[column_name]) ]
        
    return csnli_op

columns_to_normalise = ["Hinglish", "hum_gen"]


    


In [ ]:
for col in columns_to_normalise:
    traindf[col+"_csnliop"] = normalise_df_column(traindf, col)


In [ ]:
for col in columns_to_normalise:
    valdf[col+"_csnliop"] = normalise_df_column(valdf, col)


In [42]:
valdf["Hinglish_csnliop"].isna().sum()

395

In [9]:
traindf

,English,Hindi,Hinglish,Average rating,Disagreement,hum_gen,Hinglish_csnliop,hum_gen_csnliop
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6,"[Program module ek sanchika hoti hai, jisme ya...","[(module, module, en), (,, ,, univ), (ek, एक, ...","[[(Program, Program, en), (module, module, en)..."
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4,[Aur humne unke bhai Salih ko Thamud ke paas b...,"[(aur, और, hi), (hamne, हमने, hi), (aume, औमे,...","[[(Aur, और, hi), (humne, हमने, hi), (unke, उनक..."
2,"and, when reminded, do not remember\n","और जब उन्हें याद दिलाया जाता है, तो वे याद नही...","aur jab unhen yad dilaya jata hai , to ve yad ...",10,0,"[And jab remind karaya jaata hai, then they do...","[(aur, और, hi), (jab, जब, hi), (unhen, उन्हें,...","[[(And, And, en), (jab, जब, hi), (remind, remi..."
3,you won the TED Prize 2011.\n,तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n,tumhen २०११ ka ted prize mil gaya hai\n,9,1,"[tumhe TED Prize 2011 mil gaya hai.\n, You won...","[(tumhen, तुम्हें, hi), (२०११, २०११, hi), (ka,...","[[(tumhe, तुम्हें, hi), (TED, TED, acro), (Pri..."
4,He gone to Kerodemal College of Delhi Universi...,उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...,unhonne bad science karne ke lie ye delhi univ...,7,0,[Unhone baad adhyayan karne ke liye ye Delhi U...,"[(unhonne, उन्होंने, hi), (bad, bad, en), (sci...","[[(Unhone, उन्होंने, hi), (baad, बाढ़, hi), (a..."
...,...,...,...,...,...,...,...,...
2761,Polar ice caps may melt further and increase t...,अधिक मात्रा में ध्रुवों की बर्फ पिघलने से सागर...,large size men polar ki barph pighalne se ocea...,6,4,[Polar ice caps may melt further and increase ...,"[(large, large, en), (size, size, en), (men, m...","[[(Polar, Polar, en), (ice, ice, en), (caps, c..."
2762,"It ' s what turns lead into gold , and makes t...","यही चक्र सीसे को सोना बना देता है , और सोने को...","yahi chakr lead into gold bana deta hai , aur ...",7,1,[Its what turns seese into sona and makes the ...,"[(yahi, यही, hi), (chakr, चक्र, hi), (lead, le...","[[(Its, Its, en), (what, what, en), (turns, tu..."
2763,The President said the North Eastern Hill Univ...,राष्ट्रपति ने कहा कि पूर्वोत्तर पर्वतीय विश्वव...,president ne kaha ki north parvtiy university ...,8,4,[The President said the North Eastern Hill Uni...,"[(president, president, en), (ne, ने, hi), (ka...","[[(The, The, en), (President, President, en), ..."
2764,The violin bow might well have grown out of th...,बहुत संभव है कि वायलिन का गज भी एक छड़ी को दूस...,bahut snbhav hai ki vaylin ka gaj bhi ek chhar...,5,3,[The violin bow might well have grown out of t...,"[(bahut, बहुत, hi), (snbhav, संभव, hi), (hai, ...","[[(The, The, en), (violin, violin, en), (bow, ..."


In [36]:
valdf.head()

,index,English,Hindi,Hinglish,hum_gen,Hinglish_csnliop,hum_gen_csnliop,Hinglish_norm,Hinglish_lid,hum_gen_norm,...,NOUN_symcom,ADP_symcom,VERB_symcom,AUX_symcom,ADJ_symcom,SCONJ_symcom,PROPN_symcom,CCONJ_symcom,NUM_symcom,INTJ_symcom
0,0,Are you sure you want to permanently delete th...,क्या आप इन फ़ाइलों को स्थायी रूप से हटाना चाहत...,kya aap in files ko sthayi roop se permanently...,[Are you sure you want to permanently hatana c...,"[[kya, क्या, hi], [aap, आप, hi], [in, इन, hi],...","[[[Are, Are, en], [you, you, en], [sure, sure,...","[क्या, आप, इन, files, को, स्थायी, रूप, से, per...","[hi, hi, hi, en, hi, hi, hi, hi, en, en, hi]","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Three public meetings were held in Bombay City...,उस अवसर पर बंबई में तीन सभाएं की गयीं।\n,us avasar par bombay meetings were held ki gay...,[Three public meetings were held in Bombay Cit...,"[[us, उस, hi], [avasar, अवसर, hi], [par, पर, h...","[[[Three, Three, en], [public, public, en], [m...","[उस, अवसर, पर, bombay, meetings, were, held, क...","[hi, hi, hi, ne, en, en, en, hi, hi]","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Nominee of the insurance has to be a near rela...,बीमा का नामित व्यक्ति अभिदाता का निकट संबंधी ह...,insurance ka namit vyakti abhidata ka nikat sn...,[Nominee of the Bima has to be a nikat samband...,"[[insurance, insurance, en], [ka, का, hi], [na...","[[[Nominee, Nominee, en], [of, of, en], [the, ...","[insurance, का, नामित, व्यक्ति, अभिदाता, का, न...","[en, hi, hi, hi, hi, hi, hi, hi, hi]","[[वास्तव, वास्तव, वास्तव, वास्तव], [में, में, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Thus there was an inadequate appreciation of t...,इस प्रकार इस महत्वपूर्ण क्षेत्र में तेजी से का...,is prkar is vital sector inadequate appreciati...,[Is prakar ke vital chhetra me speedy action k...,"[[is, इस, hi], [prkar, प्रकार, hi], [is, इस, h...","[[[Is, इस, hi], [prakar, प्रकार, hi], [ke, के,...","[इस, प्रकार, इस, vital, sector, inadequate, ap...","[hi, hi, hi, en, en, en, en, hi, hi, hi, hi, h...","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"The Arabs laughed at him , and the alchemist l...",अरब सैनिक उसकी बात सुनकर हंस पड़े । <s> उनके स...,arab sainik uski bat sunakar hns pare arabs s...,"[The Arabs laughed at him , unke sath alchemis...","[[arab, arab, ne], [sainik, सैनिक, hi], [uski,...","[[[The, The, en], [Arabs, Arabs, ne], [laughed...","[arab, सैनिक, उसकी, बात, सुनकर, हंस, पड़े, अरब...","[ne, hi, hi, hi, hi, hi, hi, hi, hi, hi, hi, e...","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
os.getcwd()

'/home2/anmol.goel/prashantk/csnli'

In [17]:
traindf.to_json("~/prashantk/PreCogLTRC-HinglishEval-INLG-2022/dataset/traindf.json")
valdf.to_json("~/prashantk/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")

In [ ]:
traindf = pd.read_json("~/prashantk/PreCogLTRC-HinglishEval-INLG-2022/dataset/traindf.json")
valdf = pd.read_json("~/prashantk/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")


# PoS Tagging 

- uses HuggingFace's Transformer's Library
- incompatiable env with CSNLI. advisable to use seperate env for CSNLI. 

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import load_dataset, load_metric, concatenate_datasets


tokenizer = AutoTokenizer.from_pretrained("prakod/en-hi-pos-tagger-symcom")

model = AutoModelForTokenClassification.from_pretrained("prakod/en-hi-pos-tagger-symcom")


def predictposSent(model,sentence):
  
  tokenized_sentence = tokenizer(sentence,return_tensors='pt')


  mask = []
  prev_id = None
  for ind,id in enumerate(tokenized_sentence.word_ids()):
    
    if id is None:
      mask.append(-100)
    elif id == prev_id:
      mask.append(-100)
    elif id != prev_id:
      mask.append(id)
    prev_id = id


  outputs = model(**tokenized_sentence.to('cuda'))

  preds = np.argmax(outputs['logits'].cpu().detach().numpy(), axis=2).squeeze()


  true_preds = [
      label_list[p] for (p, l) in zip(preds, mask) if l != -100
  ]
  
  
  return true_preds


sent = ['abbe','keyo', 'fek', 'raha', 'hai', 'andhe', 'news', 'dalal'] 
        
sent_norm = ['अब्बी', 'क्यो', 'फेक', 'रहा', 'है', 'अंधे', 'news', 'डाल' ]

model.to('cuda')

datasets_UD = load_dataset('/home/prashantk/en-hi-pos-tagger/load_UD_enhics_mod (3).py','qhe_hiencs')

label_list = datasets_UD["train"].features[f"upos"].feature.names

tags_words = predictposSent(model,' '.join(sent))
tags_normalised = predictposSent(model,' '.join(sent_norm))

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /home/prashantk/.cache/huggingface/datasets/universal_dependencies/qhe_hiencs/0.0.0/8a3dbde569b161527eabcc186381ac424d4859ff64505fb3da8ac3e0a03c904d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
tags_words

['ADV', 'VERB', 'VERB', 'AUX', 'AUX', 'PRON', 'NOUN', 'NOUN']

In [10]:
tags_normalised

['NOUN', 'ADV', 'VERB', 'AUX', 'AUX', 'ADJ', 'NOUN', 'VERB']

In [7]:
traindf.head()

,English,Hindi,Hinglish,Average rating,Disagreement,hum_gen,Hinglish_csnliop,hum_gen_csnliop
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6,"[Program module ek sanchika hoti hai, jisme ya...","[[module, module, en], [,, ,, univ], [ek, एक, ...","[[[Program, Program, en], [module, module, en]..."
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4,[Aur humne unke bhai Salih ko Thamud ke paas b...,"[[aur, और, hi], [hamne, हमने, hi], [aume, औमे,...","[[[Aur, और, hi], [humne, हमने, hi], [unke, उनक..."
2,"and, when reminded, do not remember\n","और जब उन्हें याद दिलाया जाता है, तो वे याद नही...","aur jab unhen yad dilaya jata hai , to ve yad ...",10,0,"[And jab remind karaya jaata hai, then they do...","[[aur, और, hi], [jab, जब, hi], [unhen, उन्हें,...","[[[And, And, en], [jab, जब, hi], [remind, remi..."
3,you won the TED Prize 2011.\n,तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n,tumhen २०११ ka ted prize mil gaya hai\n,9,1,"[tumhe TED Prize 2011 mil gaya hai.\n, You won...","[[tumhen, तुम्हें, hi], [२०११, २०११, hi], [ka,...","[[[tumhe, तुम्हें, hi], [TED, TED, acro], [Pri..."
4,He gone to Kerodemal College of Delhi Universi...,उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...,unhonne bad science karne ke lie ye delhi univ...,7,0,[Unhone baad adhyayan karne ke liye ye Delhi U...,"[[unhonne, उन्होंने, hi], [bad, bad, en], [sci...","[[[Unhone, उन्होंने, hi], [baad, बाढ़, hi], [a..."


In [1]:
# extracting individual elements from csnli op


traindf["Hinglish_norm"] = [[token[1] for token in el] for el in traindf["Hinglish_csnliop"]]
traindf["Hinglish_lid"] = [[token[2] for token in el] for el in traindf["Hinglish_csnliop"]]

NameError: name 'traindf' is not defined

In [51]:
traindf["hum_gen_norm"] = [[[token[1] for sent in el] for token in sent] for el in tqdm(traindf["hum_gen_csnliop"])]
traindf["hum_gen_lid"] = [[[token[2] for sent in el] for token in sent] for el in tqdm(traindf["hum_gen_csnliop"])]

100%|██████████| 2766/2766 [00:00<00:00, 115639.78it/s]


In [49]:
hum_gen_csnliop = []
for el in traindf["hum_gen_csnliop"]:
    temp = []
    for sent in el:
        if not isinstance(sent, list):
            print(sent)
            temp.append([sent,sent,"en"])
        else:
            temp.append(sent)
    hum_gen_csnliop.append(temp)
    
traindf["hum_gen_csnliop"] = hum_gen_csnliop

In [52]:
traindf.head()

,English,Hindi,Hinglish,Average rating,Disagreement,hum_gen,Hinglish_csnliop,hum_gen_csnliop,Hinglish_norm,Hinglish_lid,hum_gen_norm,hum_gen_lid
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6,"[Program module ek sanchika hoti hai, jisme ya...","[[module, module, en], [,, ,, univ], [ek, एक, ...","[[[Program, Program, en], [module, module, en]...","[module, ,, एक, program, होती, है, ,, जिसमें, ...","[en, univ, hi, en, hi, hi, univ, hi, hi, hi, e...","[[वास्तव, वास्तव, वास्तव, वास्तव], [में, में, ...","[[hi, hi, hi, hi], [hi, hi, hi, hi], [ne, ne, ..."
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4,[Aur humne unke bhai Salih ko Thamud ke paas b...,"[[aur, और, hi], [hamne, हमने, hi], [aume, औमे,...","[[[Aur, और, hi], [humne, हमने, hi], [unke, उनक...","[और, हमने, औमे, samood, के, पास, उनके, भाई, सा...","[hi, hi, hi, ne, hi, hi, hi, hi, hi, hi, hi, h...","[[वास्तव, वास्तव, वास्तव, वास्तव], [में, में, ...","[[hi, hi, hi, hi], [hi, hi, hi, hi], [ne, ne, ..."
2,"and, when reminded, do not remember\n","और जब उन्हें याद दिलाया जाता है, तो वे याद नही...","aur jab unhen yad dilaya jata hai , to ve yad ...",10,0,"[And jab remind karaya jaata hai, then they do...","[[aur, और, hi], [jab, जब, hi], [unhen, उन्हें,...","[[[And, And, en], [jab, जब, hi], [remind, remi...","[और, जब, उन्हें, याद, दिलाया, जाता, है, ,, तो,...","[hi, hi, hi, hi, hi, hi, hi, univ, hi, en, hi,...","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...","[[hi, hi], [hi, hi], [ne, ne], [hi, hi], [en, ..."
3,you won the TED Prize 2011.\n,तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n,tumhen २०११ ka ted prize mil gaya hai\n,9,1,"[tumhe TED Prize 2011 mil gaya hai.\n, You won...","[[tumhen, तुम्हें, hi], [२०११, २०११, hi], [ka,...","[[[tumhe, तुम्हें, hi], [TED, TED, acro], [Pri...","[तुम्हें, २०११, का, red, prize, मिल, गया, है]","[hi, hi, hi, en, en, hi, hi, hi]","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...","[[hi, hi], [hi, hi], [ne, ne], [hi, hi], [en, ..."
4,He gone to Kerodemal College of Delhi Universi...,उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...,unhonne bad science karne ke lie ye delhi univ...,7,0,[Unhone baad adhyayan karne ke liye ye Delhi U...,"[[unhonne, उन्होंने, hi], [bad, bad, en], [sci...","[[[Unhone, उन्होंने, hi], [baad, बाढ़, hi], [a...","[उन्होंने, bad, science, करने, के, लिए, यह, de...","[hi, en, en, hi, hi, hi, hi, ne, en, hi, en, e...","[[वास्तव, वास्तव], [में, में], [sadar, sadar],...","[[hi, hi], [hi, hi], [ne, ne], [hi, hi], [en, ..."


In [54]:
hinglish_pos = [predictposSent(model,' '.join(sent)) for sent in tqdm(traindf["Hinglish_norm"])]

100%|██████████| 2766/2766 [00:29<00:00, 93.94it/s]


In [56]:
for normsent,pos in zip(traindf["Hinglish_norm"], hinglish_pos):
    assert len(normsent) == len(pos)

In [57]:
traindf["Hinglish_pos"] = hinglish_pos

In [58]:
hum_gen_pos = [[predictposSent(model,' '.join(sent)) for sent in el] for el in tqdm(traindf["Hinglish_norm"])]
traindf["hum_gen_pos"]  = hum_gen_pos

100%|██████████| 2766/2766 [09:02<00:00,  5.09it/s]


In [62]:
traindf["hum_gen_pos"]  = hum_gen_pos

In [ ]:
valdf["Hinglish_norm"] = [[token[1] for token in el] for el in valdf["Hinglish_csnliop"]]
valdf["Hinglish_lid"] = [[token[2] for token in el] for el in valdf["Hinglish_csnliop"]]

valdf["hum_gen_norm"] = [[[token[1] for sent in el] for token in sent] for el in tqdm(valdf["hum_gen_csnliop"])]
valdf["hum_gen_lid"] = [[[token[2] for sent in el] for token in sent] for el in tqdm(valdf["hum_gen_csnliop"])]

In [34]:
from collections import Counter

Counter([type(el) for el in valdf["Hinglish_norm"]])

Counter({list: 395, float: 395})

In [ ]:
valdf["Hinglish_pos"] = [predictposSent(model,' '.join(sent)) for sent in tqdm(valdf["Hinglish_norm"])]
 

In [31]:
valdf["Hinglish_pos"].isna().sum()

395

In [13]:
valdf["Hinglish_pos"]

0      [PRON, PRON, DET, NOUN, ADP, ADV, PART, PART, ...
1       [DET, NOUN, ADP, ADJ, NOUN, AUX, VERB, AUX, AUX]
2      [NOUN, ADP, ADJ, NOUN, NOUN, ADP, ADJ, PART, AUX]
3      [DET, NOUN, DET, ADJ, NOUN, ADJ, NOUN, ADP, NO...
4      [PROPN, NOUN, PRON, NOUN, VERB, VERB, AUX, PRO...
                             ...                        
390    [SCONJ, PROPN, NOUN, ADP, NOUN, ADP, VERB, ADJ...
391                               [DET, NOUN, ADP, PART]
392    [ADJ, NOUN, ADP, ADV, ADJ, NOUN, ADP, ADP, ADP...
393    [DET, NOUN, ADP, ADP, ADP, ADP, NOUN, NOUN, VE...
394      [PRON, NOUN, PROPN, ADP, NOUN, NOUN, ADJ, NOUN]
Name: Hinglish_pos, Length: 395, dtype: object

In [61]:
valdf["hum_gen_pos"] = [[predictposSent(model,' '.join(sent)) for sent in el] for el in tqdm(valdf["Hinglish_norm"])]

100%|██████████| 395/395 [01:16<00:00,  5.18it/s]


In [ ]:
traindf.to_json("/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/traindf.json")
valdf.to_json("/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")

In [3]:
traindf = pd.read_json("/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/traindf.json")
valdf = pd.read_json("/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")

In [ ]:
sys.path.append('/home2/anmol.goel/prashantk/cs_metrics')

from cs_metrics import *

sample = 'EN EN HI HI UNIV UNIV HI HI EN EN EN HI HI'
cmi(sample)


In [6]:
traindf.head(), traindf.columns

(                                             English  \
 0  Program module is a file that contains instruc...   
 1  And to Thamud We sent their brother Sali 'h. H...   
 2              and, when reminded, do not remember\n   
 3                      you won the TED Prize 2011.\n   
 4  He gone to Kerodemal College of Delhi Universi...   
 
                                                Hindi  \
 0  माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...   
 1  और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...   
 2  और जब उन्हें याद दिलाया जाता है, तो वे याद नही...   
 3          तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n   
 4  उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...   
 
                                             Hinglish  Average rating  \
 0  module , ek program hoti hai , jismen ya to so...               7   
 1  aur hamne aume samood ke pas unke bhaee saleh ...               6   
 2  aur jab unhen yad dilaya jata hai , to ve yad ...              10   
 3            tumhen

In [8]:
cmi_scores = []

for ind, row in tqdm(traindf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]

    
    # cmi_score = cmi(' '.join(lid))

        
    try:
        cmi_score = cmi(' '.join(lid))
    
    except IndexError as e:
        print(lid,e)
        cmi_score = 0
    
    cmi_scores.append(cmi_score)

    
traindf['Hinglish_cmi_scores'] = cmi_scores

2766it [00:00, 8255.35it/s]


In [10]:
def spavg_test(x, k= 2):
        
    LANG_TAGS = ['EN', 'HI']
    OTHER_TAGS = ['UNIV', 'NE','ACRO']

    LANG_TAGS = [tag.lower() for tag in LANG_TAGS]
    OTHER_TAGS = [tag.lower() for tag in OTHER_TAGS]

    if isinstance(x,str):
        x = x.split()
        
    x = [el.lower() for el in x]

    count = 0 
    mem = None
    for l_i, l_j in zip(x,x[1:]):
        if l_i in OTHER_TAGS:
            continue
        if l_i != l_j:
            count+=1

    return count 


sp_scores = []

for ind, row in tqdm(traindf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
        
    try:
        score = spavg_test(' '.join(lid))
    
    except IndexError as e:
        print(lid,e)
        score = np.nan
    
    sp_scores.append(score)

2766it [00:00, 9703.44it/s]


In [11]:
burstiness_scores = []

exceptionCount, IndexErrorCount, bothEnHiPresent = 0,0,0

for ind, row in tqdm(traindf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
        
    try:
        score = burstiness(' '.join(lid))
    
    except IndexError as e:
        # print(lid,e)
        score = np.nan
        IndexErrorCount +=1
        
    except Exception as e:
        # print(lid, e)
        score = np.nan
        c = Counter(lid)
        if 'EN' in c and 'HI' in c:
            bothEnHiPresent+=1
        exceptionCount +=1
    
    burstiness_scores.append(score)

print(exceptionCount, bothEnHiPresent, IndexErrorCount)

2766it [00:00, 4527.79it/s]

256 41 0


In [12]:
traindf['Hinglish_burstiness_scores'] = burstiness_scores
traindf['Hinglish_sp_scores'] = sp_scores

In [27]:
class CodeMIxSentence():
    
    def __init__(self, sentence = None,
                         tokens = None,
                         LID_Tags = None,
                         PoS_Tags = None):
        
        self.sentence = sentence
        self.tokens = tokens
        self.LID_Tags = LID_Tags
        self.PoS_Tags = PoS_Tags
        self.length = len(LID_Tags) # has to changed to take length from number of token
        
        self.LID_count_map = dict(Counter(LID_Tags).most_common())
        self.PoS_count_map = dict(Counter(PoS_Tags).most_common())      
        
        lid_pos_combined = []
        
        for lid,pos in zip(self.LID_Tags, self.PoS_Tags):
            
            lid_pos_combined.append(pos+'_'+lid)
            
        self.LID_POS_count_map = dict(Counter(lid_pos_combined).most_common())


class SyMCoM:
    
    def __init__(self, 
                LID_tagset = None,
                PoS_tagset = None, 
                L1 = None, 
                L2 = None):
        
        self.LID_tagset = LID_tagset
        self.PoS_tagset = PoS_tagset
        self.l1 = L1
        self.l2 = L2
        
        
    def symcom_pos_tags(self, codemixsent_obj):
        
        symcom_scores_pos_tags = {}
        
        l1 = self.l1
        l2 = self.l2
        

            
                
        for postag in codemixsent_obj.PoS_count_map:
            
            if postag not in ['PUNCT','SYM', 'X']:
            
                pos_l1 = codemixsent_obj.LID_POS_count_map[postag+'_'+self.l1] if postag+'_'+self.l1 in codemixsent_obj.LID_POS_count_map else 0
                pos_l2 = codemixsent_obj.LID_POS_count_map[postag+'_'+self.l2] if postag+'_'+self.l2 in codemixsent_obj.LID_POS_count_map else 0
                
                try:
                    symcom_scores_pos_tags[postag+'_symcom'] = (pos_l1 - pos_l2) / (pos_l1 + pos_l2) 
                    
                except ZeroDivisionError: # to handle cases where there the LID for pos tag is not en or hi. 
                    pass
        return symcom_scores_pos_tags
    
    
        
        
    def symcom_sentence(self, codemixsent_obj):
        
        symcom_sentence = 0
        
        symcom_scores_pos_tags = self.symcom_pos_tags(codemixsent_obj)
        
        for pos, score in symcom_scores_pos_tags.items():
            pos = pos.split('_')[0]
            symcom_sentence += abs(score) * (codemixsent_obj.PoS_count_map[pos] / codemixsent_obj.length)     
            
        return symcom_sentence

In [29]:
symcom = SyMCoM(L1 = 'en',
                L2 = 'hi', 
                LID_tagset = ['hi', 'en', 'ne', 'univ', 'acro'],
                PoS_tagset = ['NOUN', 'ADV', 'VERB', 'AUX', 'ADJ', 'ADP', 'PUNCT', 'DET', 'PRON', 'PROPN', 'PART', 'CCONJ', 'SCONJ', 'INTJ', 'NUM', 'SYM','X'])



In [ ]:
symcom = SyMCoM(L1 = 'en',
                L2 = 'hi', 
                LID_tagset = ['hi', 'en', 'ne', 'univ', 'acro'],
                PoS_tagset = ['NOUN', 'ADV', 'VERB', 'AUX', 'ADJ', 'ADP', 'PUNCT', 'DET', 'PRON', 'PROPN', 'PART', 'CCONJ', 'SCONJ', 'INTJ', 'NUM', 'SYM','X'])


symcom_scores_pos, symcom_sent = [],[]

div_by_zero_error_count, errors  = 0 , []

for ind, row in tqdm(traindf.iterrows()):

    cm_sentence = CodeMIxSentence(sentence = None,
                                     tokens = row['Hinglish_norm'],
                                     LID_Tags = row['Hinglish_lid'],
                                     PoS_Tags = row['Hinglish_pos'])
    symcom_scores_pos.append(symcom.symcom_pos_tags(cm_sentence))
    symcom_sent.append(symcom.symcom_sentence(cm_sentence))

In [15]:
traindf['symcom_scores_pos'] = symcom_scores_pos
traindf['symcom_scores_sent'] = symcom_sent

In [16]:
cmi_scores = []

for ind, row in tqdm(valdf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]

    
    # cmi_score = cmi(' '.join(lid))

        
    try:
        cmi_score = cmi(' '.join(lid))
    
    except IndexError as e:
        print(lid,e)
        cmi_score = 0
    
    cmi_scores.append(cmi_score)

    
valdf['Hinglish_cmi_scores'] = cmi_scores

395it [00:00, 7480.63it/s]


In [17]:
def spavg_test(x, k= 2):
        
    LANG_TAGS = ['EN', 'HI']
    OTHER_TAGS = ['UNIV', 'NE','ACRO']

    LANG_TAGS = [tag.lower() for tag in LANG_TAGS]
    OTHER_TAGS = [tag.lower() for tag in OTHER_TAGS]

    if isinstance(x,str):
        x = x.split()
        
    x = [el.lower() for el in x]

    count = 0 
    mem = None
    for l_i, l_j in zip(x,x[1:]):
        if l_i in OTHER_TAGS:
            continue
        if l_i != l_j:
            count+=1

    return count 


sp_scores = []

for ind, row in tqdm(valdf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
        
    try:
        score = spavg_test(' '.join(lid))
    
    except IndexError as e:
        print(lid,e)
        score = np.nan
    
    sp_scores.append(score)
    
burstiness_scores = []

exceptionCount, IndexErrorCount, bothEnHiPresent = 0,0,0

for ind, row in tqdm(valdf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
        
    try:
        score = burstiness(' '.join(lid))
    
    except IndexError as e:
        # print(lid,e)
        score = np.nan
        IndexErrorCount +=1
        
    except Exception as e:
        # print(lid, e)
        score = np.nan
        c = Counter(lid)
        if 'EN' in c and 'HI' in c:
            bothEnHiPresent+=1
        exceptionCount +=1
    
    burstiness_scores.append(score)

print(exceptionCount, bothEnHiPresent, IndexErrorCount)    

395it [00:00, 9458.28it/s]
395it [00:00, 4566.94it/s]

32 2 0


In [18]:
valdf['Hinglish_burstiness_scores'] = burstiness_scores
valdf['Hinglish_sp_scores'] = sp_scores

In [ ]:
symcom_scores_pos, symcom_sent = [],[]

div_by_zero_error_count, errors  = 0 , []

for ind, row in tqdm(valdf.iterrows()):

    cm_sentence = CodeMIxSentence(sentence = None,
                                     tokens = row['Hinglish_norm'],
                                     LID_Tags = row['Hinglish_lid'],
                                     PoS_Tags = row['Hinglish_pos'])
    symcom_scores_pos.append(symcom.symcom_pos_tags(cm_sentence))
    symcom_sent.append(symcom.symcom_sentence(cm_sentence))
    
valdf['symcom_scores_pos'] = symcom_scores_pos
valdf['symcom_scores_sent'] = symcom_sent    

In [20]:
traindf.to_json("~/prashantk/PreCogLTRC-HinglishEval-INLG-2022/dataset/traindf.json")
valdf.to_json("~/prashantk/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")

In [2]:
valdf = pd.read_json("~/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")

In [14]:
valdf.columns

Index(['English', 'Hindi', 'Hinglish', 'hum_gen', 'Hinglish_csnliop',
       'hum_gen_csnliop', 'Hinglish_norm', 'Hinglish_lid', 'hum_gen_norm',
       'hum_gen_lid', 'Hinglish_pos', 'hum_gen_pos', 'Hinglish_cmi_scores',
       'Hinglish_burstiness_scores', 'Hinglish_sp_scores', 'symcom_scores_pos',
       'symcom_scores_sent'],
      dtype='object')

In [20]:
df2 = pd.json_normalize(traindf["symcom_scores_pos"])
traindf = pd.concat([traindf,df2], axis = 1)
print(traindf.shape)
traindf.reset_index(inplace = True)

traindf.shape

In [ ]:
df2 = pd.json_normalize(valdf["symcom_scores_pos"])
valdf = pd.concat([valdf,df2], axis = 1)
valdf.reset_index(inplace = True)

valdf.shape

In [21]:
valdf.columns

Index(['index', 'English', 'Hindi', 'Hinglish', 'hum_gen', 'Hinglish_csnliop',
       'hum_gen_csnliop', 'Hinglish_norm', 'Hinglish_lid', 'hum_gen_norm',
       'hum_gen_lid', 'Hinglish_pos', 'hum_gen_pos', 'Hinglish_cmi_scores',
       'Hinglish_burstiness_scores', 'Hinglish_sp_scores', 'symcom_scores_pos',
       'symcom_scores_sent', 'symcom_+1_count', 'symcom_-1_count',
       'symcom_na_count', 'symcom_others', 'PRON_symcom', 'ADV_symcom',
       'PART_symcom', 'DET_symcom', 'NOUN_symcom', 'ADP_symcom', 'VERB_symcom',
       'AUX_symcom', 'ADJ_symcom', 'SCONJ_symcom', 'PROPN_symcom',
       'CCONJ_symcom', 'NUM_symcom', 'INTJ_symcom'],
      dtype='object')

In [10]:
valdf["symcom_scores_pos"].head()

0    {'PRON_symcom': -1.0, 'ADV_symcom': 0.0, 'PART...
1    {'AUX_symcom': -0.33333333330000003, 'NOUN_sym...
2    {'NOUN_symcom': -0.33333333330000003, 'ADP_sym...
3    {'NOUN_symcom': -0.33333333330000003, 'DET_sym...
4    {'PROPN_symcom': -1.0, 'NOUN_symcom': -0.33333...
Name: symcom_scores_pos, dtype: object

In [24]:
def count_plusone(sample):
    count = 0
    try:
        assert isinstance(sample,dict)
    except AssertionError:
        return np.nan
    for k,v in sample.items():
        if v == 1:
            count+=1
    return count

def count_minus1(sample):
    print(sample)
    count = 0
    for k,v in sample.items():
        if v == -1:
            count+=1
    return count

def countNaN(sample):
    count = 14 - len(sample.keys()) 
    return count

In [25]:
valdf["symcom_+1_count"] = valdf["symcom_scores_pos"].apply(count_plusone)
valdf["symcom_-1_count"] = valdf["symcom_scores_pos"].apply(count_minus1)
valdf["symcom_na_count"] = valdf["symcom_scores_pos"].apply(countNaN)
valdf["symcom_others"] = valdf.apply(lambda row : 14 - row["symcom_+1_count"] - row["symcom_-1_count"]- row["symcom_na_count"] , axis = 1)    


{'PRON_symcom': -1.0, 'ADV_symcom': 0.0, 'PART_symcom': -1.0, 'DET_symcom': -1.0, 'NOUN_symcom': 1.0, 'ADP_symcom': -1.0, 'VERB_symcom': 1.0, 'AUX_symcom': -1.0}
{'AUX_symcom': -0.33333333330000003, 'NOUN_symcom': 0.0, 'DET_symcom': -1.0, 'ADP_symcom': -1.0, 'VERB_symcom': 1.0}
{'NOUN_symcom': -0.33333333330000003, 'ADP_symcom': -1.0, 'ADJ_symcom': -1.0, 'PART_symcom': -1.0, 'AUX_symcom': -1.0}
{'NOUN_symcom': -0.33333333330000003, 'DET_symcom': -1.0, 'ADJ_symcom': 1.0, 'ADP_symcom': -1.0, 'VERB_symcom': -1.0, 'SCONJ_symcom': -1.0, 'PART_symcom': -1.0, 'AUX_symcom': -1.0}
{'PROPN_symcom': -1.0, 'NOUN_symcom': -0.33333333330000003, 'VERB_symcom': -1.0, 'PRON_symcom': -1.0, 'AUX_symcom': -1.0, 'ADP_symcom': -1.0, 'PART_symcom': -1.0}
{'AUX_symcom': -0.6666666667000001, 'ADJ_symcom': -0.33333333330000003, 'VERB_symcom': -1.0, 'NOUN_symcom': 0.0, 'DET_symcom': -1.0, 'CCONJ_symcom': -1.0}
{'NOUN_symcom': -0.7142857143, 'VERB_symcom': -1.0, 'AUX_symcom': -1.0, 'DET_symcom': -1.0, 'PRON_symco

AttributeError: 'float' object has no attribute 'items'

In [18]:
valdf.to_json("~/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")

In [ ]:

traindf["symcom_+1_count"] = traindf["symcom_scores_pos"].apply(count_plusone)
traindf["symcom_-1_count"] = traindf["symcom_scores_pos"].apply(count_minus1)
traindf["symcom_na_count"] = traindf["symcom_scores_pos"].apply(countNaN)
traindf["symcom_others"] = traindf.apply(lambda row : 14 - row["symcom_+1_count"] - row["symcom_-1_count"]- row["symcom_na_count"] , axis = 1)




In [ ]:
traindf.to_json("/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/traindf.json")
valdf.to_json("/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/valdf.json")